# Q2: ViT for classification

## data preprocess

In [4]:
import os
from PIL import Image

images = os.listdir("./data/BCCD_Dataset/BCCD/JPEGImages/")

img_path = os.path.join("./data/BCCD_Dataset/BCCD/JPEGImages/", images[0])
img = Image.open(img_path)
img.show()